# 번역

## 규칙 기반 기계 번역(RBMT, Rule-Based Machine Translation)
    - 번역할때 경우의 수를 직접 정의해 주는 방식(1954년, 미국의 조지타운대와 IBM이 직접 정의한 언어 규칙을 통해 60개의 러시아어 문장을 영어로 번역하는데 성공
    
    - 단점: 규칙에 없는 문장이 들어올 경우 번역이 불가능하고 유연하게 번역하지 못하며, 무엇보다 모든 규칙을 정의하는 과정이 너무 복잡하고 오랜 시간이 필요하다.

## 통계적 기반 번역
    - 무수한 데이터로부터 통계적 확률을 구해 번역을 진행하는 통계적 기계번역

## Beam Search
    - 지금 상황에서 가장 높은 확률을 갖는 Top-k 문장만 쟁취
    - Beam Size는 연산량과 성능 간의 Trade-off관계를 가지고 있다. 컴퓨터 성능만 좋다면 Beam size가 크면 클수록 성능이 좋아진다.

## Beam Search 코드

In [1]:
import math
import numpy as np

def beam_search_decoder(prob, beam_size):
    sequences = [[[], 1.0]]  # 생성된 문장과 점수를 저장

    for tok in prob:
        all_candidates = []

        for seq, score in sequences:
            for idx, p in enumerate(tok): # 각 단어의 확률을 총점에 누적 곱
                candidate = [seq + [idx], score * -math.log(-(p-1))]
                all_candidates.append(candidate)

        ordered = sorted(all_candidates,
                         key=lambda tup:tup[1],
                         reverse=True) # 총점 순 정렬
        sequences = ordered[:beam_size] # Beam Size에 해당하는 문장만 저장 

    return sequences

In [2]:
vocab = {
    0: "<pad>",
    1: "까요?",
    2: "커피",
    3: "마셔",
    4: "가져",
    5: "될",
    6: "를",
    7: "한",
    8: "잔",
    9: "도",
}

# prob_seq은 문장의 각 위치에서 어떤 단어가 생성될지의 확률을 한 번에 정의해둔 것입니다.
# 실제로는 각 단어에 대한 확률이 prob_seq처럼 한번에 정의되지 않기 때문에 실제 문장 생성과정과는 거리가 멉니다.
# 하지만 Beam Search의 동작과정 이해를 돕기위해 이와 같은 예시를 보여드립니다.
# prob_seq의 각 열은 위 vocab의 각 숫자(key)에 대응됩니다.
prob_seq = [[0.01, 0.01, 0.60, 0.32, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01], # 커피 : 0.60
            [0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.75, 0.01, 0.01, 0.17], # 를 : 0.75
            [0.01, 0.01, 0.01, 0.35, 0.48, 0.10, 0.01, 0.01, 0.01, 0.01], # 가져 : 0.48
            [0.24, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.68], # 도 : 0.68
            [0.01, 0.01, 0.12, 0.01, 0.01, 0.80, 0.01, 0.01, 0.01, 0.01], # 될 : 0.80
            [0.01, 0.81, 0.01, 0.01, 0.01, 0.01, 0.11, 0.01, 0.01, 0.01], # 까요? : 0.81
            [0.70, 0.22, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01], # <pad> : 0.91
            [0.91, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01], # <pad> : 0.91
            [0.91, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01], # <pad> : 0.91
            [0.91, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01]] # <pad> : 0.91

prob_seq = np.array(prob_seq)
beam_size = 3

result = beam_search_decoder(prob_seq, beam_size)

for seq, score in result:
    sentence = ""

    for word in seq:
        sentence += vocab[word] + " "

    print(sentence, "// Score: %.4f" % score)

커피 를 가져 도 될 까요? <pad> <pad> <pad> <pad>  // Score: 42.5243
커피 를 마셔 도 될 까요? <pad> <pad> <pad> <pad>  // Score: 28.0135
마셔 를 가져 도 될 까요? <pad> <pad> <pad> <pad>  // Score: 17.8983


In [3]:
# beam_size를 더키움
beam_size = 30

result = beam_search_decoder(prob_seq, beam_size)

for seq, score in result:
	sentence = ""
	
	for word in seq:	
		sentence += vocab[word] + " "

	print(sentence, "// Score: %.4f" % score)

커피 를 가져 도 될 까요? <pad> <pad> <pad> <pad>  // Score: 42.5243
커피 를 마셔 도 될 까요? <pad> <pad> <pad> <pad>  // Score: 28.0135
마셔 를 가져 도 될 까요? <pad> <pad> <pad> <pad>  // Score: 17.8983
마셔 를 마셔 도 될 까요? <pad> <pad> <pad> <pad>  // Score: 11.7907
커피 를 가져 <pad> 될 까요? <pad> <pad> <pad> <pad>  // Score: 10.2421
커피 를 가져 도 될 까요? 까요? <pad> <pad> <pad>  // Score: 8.7757
커피 를 될 도 될 까요? <pad> <pad> <pad> <pad>  // Score: 6.8515
커피 를 마셔 <pad> 될 까요? <pad> <pad> <pad> <pad>  // Score: 6.7471
커피 를 마셔 도 될 까요? 까요? <pad> <pad> <pad>  // Score: 5.7811
커피 도 가져 도 될 까요? <pad> <pad> <pad> <pad>  // Score: 5.7156
마셔 를 가져 <pad> 될 까요? <pad> <pad> <pad> <pad>  // Score: 4.3109
커피 도 마셔 도 될 까요? <pad> <pad> <pad> <pad>  // Score: 3.7652
마셔 를 가져 도 될 까요? 까요? <pad> <pad> <pad>  // Score: 3.6936
커피 를 가져 도 커피 까요? <pad> <pad> <pad> <pad>  // Score: 3.3776
커피 를 가져 도 될 를 <pad> <pad> <pad> <pad>  // Score: 2.9839
마셔 를 될 도 될 까요? <pad> <pad> <pad> <pad>  // Score: 2.8838
마셔 를 마셔 <pad> 될 까요? <pad> <pad> <pad> <pad>  // Score: 2.8398
마셔

## sampling 기법
    - 다음의 단어로 나올 확률이 높은 단어를 선택(확률적으로 단어를 뽑는 방법)
    - 높은 확률을 갖는 단어를 택하는 경우가 가장 많기 때문에 랜덤이지만 뜬금없는 문장이 생성될 확률은 낮다

## Data-Augmentation
    - '데이터 증가', '데이터 확대'등으로 해석되며 훈련 데이터를 수십 배까지도 부풀리는 기술

## Data-Augmentation의 종류들

## Lexical-Substitution(어휘 대체)
    - 동의어 기반 대체(시소러스: 어떤 단어의 동의어나 유의어를 집중적으로 구축해 놓은 사전
    

## Embedding 활용 대체
    - Pre-training World Embedding(Word2Vec, GloVe)

## Ranom Noise Injection
    - 오타 노이즈를 추가하는것
    ex) '올 때 아이스크림 사와', '놀 때 아이스크림 사와'로 바꾸는 등 약간의 오타를 추가 하는 것 
    - 공백 노이즈 추가
    ex) 공부를 하는 것은 자연스러운 일이다.
        공부를 하는 __은 자연스러운 일이다. 
    (문장의 일부 단어를 공백 토큰으로 치환, 학습의 과적합을 방지하는 데에 좋은 효과)
    - 랜덤 유의어 추가
    주어진 문장에서 불용어(Stop word)가 아닌 단어를 랜덤하게 뽑은 후,해당 단어와 유사한 단어를 골라 문장에 아무렇게나 삽입
    오늘은 왠지 아주 신나요 -> 오늘은 왠지 행복 아주 신나요(원본의 단어가 손실되지 않는다는 것)

## 채점
    - BLEU(Bilingual Evaluation Understudy) Score(평가지표)
    : '기계가 실제 번역을 얼마나 잘 구현하는가?'를 평가
    공식(BLEU = min(1, 예측문장/실제문장)

## 챗봇

## 모델본류(검색기반모델 or 생성모델)

## 검색기반 모델(Retrieval-based models)
    -: 미리 정해진 응답들의 데이터를 사용하며, 인풋과 문맥에 기반하여 적절한 응답을 휴리스틱적 방법
    (휴리스틱 방법은 규칙기반으로 단순하고, 머신 러닝 분류기의 조화만큼 복잡해짐)

## 생성 모델(Generative models)
    - 미리 정해둔 응답에 의존하지 않는다. 스크래치부터 새로운 응답을 생성한다.

## 챗봇 설명
    - 챗봇은 대화를 목적으로하는 모델(질문만 하는 언어를 Source 언어, 답변만 하는 언어를 Target) --> source 언어의 Embedding 공간 속 문장을 Target언어의 Embedding공간으로 매핑